In [1]:
import torch
from torch import nn
from torch import optim
from torchinfo import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from pathlib import Path
import wandb
import sys
import os

sys.path.append("../src")

from trainer import Trainer
from module import ResNetModule
from utils import model_size, load_from_checkpoint
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger
from dataset import MapDataset
from models import ResNet18
import settings as s

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name=s.project_name,
    config={
        "model": s.model,
        "dataset": s.dataset,
        "max_epochs": s.max_epochs,
        "optimizer": s.optimizer,
        "lr_scheduler": s.lr_scheduler,
        "test_run": s.test_run,
        "transfer_learning": s.transfer_learning
    },
    logs_path=logs_path,
    offline=s.wandb_offline
)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# cpu_count = os.cpu_count()
cpu_count = 7

dataset = CIFAR10(data_path, train=True, download=True)

train_dataset, val_dataset = random_split(
    dataset, [s.dataset["train_split"], s.dataset["val_split"]]
)

normalize_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_transforms = v2.Compose([
    v2.RandomCrop(size=(32, 32), padding=4, padding_mode='reflect'),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(p=0.2),
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    v2.RandomRotation(degrees=15),
    v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    v2.RandomGrayscale(p=0.1),

    normalize_transforms
])

val_transforms = normalize_transforms

train_dataset = MapDataset(train_dataset, transform=train_transforms)
val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=s.dataset["batch_size"], shuffle=True, num_workers=cpu_count)
val_dataloader = DataLoader(
    val_dataset, batch_size=s.dataset["batch_size"],  num_workers=cpu_count)

Files already downloaded and verified


In [5]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=90.0, accuracy_diff=5.0, wait_epochs=5),
    # OverfitCallback(limit_batches=1, batch_size=10, max_epochs=500, augument_data=False)
]

In [6]:
api = wandb.Api()
run = api.run("sampath017/ImageClassification/24z2beff")
artifact = api.artifact('sampath017/ImageClassification/run-24z2beff-best_val_acc_68.66.pt:v0', type='model')
local_path = artifact.download(root=logs_path)
checkpoint = torch.load(Path(local_path)/"best_val_acc_68.66.pt", weights_only=True, map_location=device)

model = ResNet18(num_classes=100)
model.load_state_dict(checkpoint["model"])
summary(
    model,
    input_size=(s.dataset["batch_size"],
                *train_dataset[0][0].shape),
    device="cpu",
    mode="train",
    depth=1
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Downloading large artifact run-24z2beff-best_val_acc_68.66.pt:v0, 319.00MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.0


Layer (type:depth-idx)                             Output Shape              Param #
ResNet18                                           [512, 100]                --
├─Sequential: 1-1                                  [512, 512, 1, 1]          27,811,392
├─Sequential: 1-2                                  [512, 100]                51,300
Total params: 27,862,692
Trainable params: 27,862,692
Non-trainable params: 0
Total mult-adds (Units.TERABYTES): 1.41
Input size (MB): 6.29
Forward/backward pass size (MB): 8925.89
Params size (MB): 111.45
Estimated Total Size (MB): 9043.63

In [7]:
for params in model.parameters():
    params.requires_grad = False 

model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512, 10),
)

summary(
    model,
    input_size=(s.dataset["batch_size"],
                *train_dataset[0][0].shape),
    device="cpu",
    mode="train",
    depth=1
)

In [8]:
module = ResNetModule(model)

optimizer = optim.AdamW(
    params=module.model.parameters(),
    weight_decay=s.optimizer["weight_decay"]
)

try:
    if s.lr_scheduler["name"] == "OneCycleLR":
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer,
            max_lr=s.lr_scheduler["max_lr"],
            epochs=s.max_epochs,
            steps_per_epoch=len(train_dataloader),
        )

        print(s.lr_scheduler["name"])
except TypeError:
    lr_scheduler = None
    print("lr_scheduler is None!")

OneCycleLR


In [9]:
trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=s.fast_dev_run,
    limit_batches=s.limit_batches,
    lr_scheduler=lr_scheduler,
    save_checkpoint_type="best_val",
    num_workers=cpu_count
)

Using device: cuda!


In [10]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Time per epoch: 20.38 seconds
Epoch: 0, train_accuracy: 47.56, val_accuracy: 66.90, lr: 0.0004
Epoch: 1, train_accuracy: 65.01, val_accuracy: 71.05, lr: 0.0005
Epoch: 2, train_accuracy: 67.74, val_accuracy: 73.10, lr: 0.0008
Epoch: 3, train_accuracy: 69.75, val_accuracy: 74.82, lr: 0.0013
Epoch: 4, train_accuracy: 71.08, val_accuracy: 76.10, lr: 0.0020
Epoch: 5, train_accuracy: 71.63, val_accuracy: 77.44, lr: 0.0028
Epoch: 6, train_accuracy: 72.57, val_accuracy: 78.21, lr: 0.0037
Epoch: 7, train_accuracy: 72.78, val_accuracy: 78.36, lr: 0.0047
Epoch: 8, train_accuracy: 73.47, val_accuracy: 78.33, lr: 0.0057
Epoch: 9, train_accuracy: 73.57, val_accuracy: 78.42, lr: 0.0067
Epoch: 10, train_accuracy: 73.34, val_accuracy: 78.22, lr: 0.0076
Epoch: 11, train_accuracy: 73.39, val_accuracy: 78.29, lr: 0.0084
Epoch: 12, train_accuracy: 73.44, val_accuracy: 78.49, lr: 0.0091
Epoch: 13, train_accuracy: 73.44, val_accuracy: 78.70, lr: 0.0096
Epoch: 14, train_accuracy: 74.03, val_accuracy: 78.50, l

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
epoch_train_accuracy,▁▅▆▆▇▇▇▇▇▇▇▇▇█▇█████████████████████████
epoch_train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_val_accuracy,▁▃▄▅▆▇▇▇▇▇▇▇▇▇███▇▇██▇█▇██▇█████████████
epoch_val_loss,█▄▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▁▂▂▃▄▅▆▆▇▇▇▇████████▇▇▇▇▇▆▅▅▅▄▄▄▄▃▃▂▂▁▁▁
step_train_accuracy,▁▅▄▆▅▇▆█▆▇▇▇█▆▇▇█▆▆▇▆▆█▅▇█▆▇▆▇▇▇▇▇▇▇▇██▆
step_train_loss,█▆▆▅▅▂▄▄▂▂▂▂▃▃▃▃▃▃▄▂▃▂▂▄▂▂▂▃▃▃▂▁▁▂▃▂▂▃▃▂
step_val_accuracy,▁▃▄▅▆▃▇▆▆▇▆▇▆▇▇▆▇▇▆▇▆█▇▇▇▇▇▆▇▆▅▇█▆▆▇█▇▆▆
step_val_loss,██▅▄▃▃▃▃▃▂▂▂▂▂▂▂▃▃▂▂▃▂▃▃▂▁▁▁▂▃▂▂▃▃▁▃▂▃▃▂
training_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇████


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)